To use this notebook, you can install [`iJavascript`](https://github.com/n-riesco/ijavascript).

In [ ]:
AUTH_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAJv13AAAAAAAr%2FT44AfHGR071THYE%2FC8pBwpNn8%3Dnp0S2OPJWEpTAaXa9A3Y2WC3BH6VMZf1WYVRI0GCMhgtKNGfDs';
TwitterApi = require('twitter-api-v2').default
client = new TwitterApi(AUTH_TOKEN)

In [ ]:
quoterot = '1460777590373769226'
snorlax = '1423685171430178816'
phallusoraptors = '1310573765319946240'
altyatharth = '1340667684363628545'
askyatharth = '193553265'

## v1 API

In [4]:
function getRateLimitStatus(client, endpoint) {
    const lastStatus = client.v1.getLastRateLimitStatus(endpoint);
    return {
        'known': !!lastStatus,
        'obsolete': client.v1.isRateLimitStatusObsolete(endpoint),
        'hit': client.v1.hasHitRateLimit(endpoint),
        ...lastStatus,
    }
}

In [5]:
async function getFollowers(client, username) {
    response = await client.v1.get('followers/ids.json', {'screen_name': username, 'stringify_ids': true})
    return response['ids']
}

In [ ]:
followerIds = await getFollowers(client, 'quoterot')

In [ ]:
tweet = await client.v1.singleTweet('1461964744064057352')
tweet.is_quote_status
tweet.quoted_status_id_str
tweet.quoted_status_permalink.expanded

In [ ]:
userTimeline = await client.v1.userTimeline(altyatharth)
tweets = userTimeline.tweets
oldest_tweet = tweets.at(-1)
oldest_tweet_date = new Date(oldest_tweet['created_at'])
quote_tweets = tweets.filter((tweet) => tweet.is_quote_status)
quoted_tweets = quote_tweets.map((tweet) => tweet.quoted_status_id_str)
quoted_tweets = quote_tweets.map((tweet) => tweet.quoted_status_permalink.expanded)

## v2 API

In [ ]:
TwitterApi = require('twitter-api-v2').default
axios = require('axios')

AUTH_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAMj2VwEAAAAAxhAqlNP8HzK8uzmSmvAO5LhzNcA%3D8iuwR8sYY3jcfeGIlJ8dThKv7zoU0DhBqQArw3elaTD2mF73Ko'
client = new TwitterApi(AUTH_TOKEN)

In [ ]:
quoterot = '1460777590373769226'
snorlax = '1423685171430178816'
phallusoraptors = '1310573765319946240'
altyatharth = '1340667684363628545'
askyatharth = '193553265'

In [ ]:
quoterot = (await client.v2.userByUsername('quoterot')).data.id

In [9]:
function getRateLimitStatus(client, endpoint) {
    const lastStatus = client.v2.getLastRateLimitStatus(endpoint);
    return {
        'known': !!lastStatus,
        'obsolete': client.v2.isRateLimitStatusObsolete(endpoint),
        'hit': client.v2.hasHitRateLimit(endpoint),
        ...lastStatus,
    }
}

In [ ]:
getRateLimitStatus(client, 'statuses/user_timeline.json')

In [10]:
async function getTweetsByIds(ids) {
    options = {'tweet.fields': 'in_reply_to_user_id,referenced_tweets', 'expansions': 'in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id'}
    return await client.v2.tweets(ids, options)
}

// await client.v2.get('tweets/:id', options, { params: { id: '1461964695556943879' }, fullResponse: true })

async function getTweetByUrl(url) {
    tweet_id = url.split('/').at(-1).split('?')[0]
    return getTweetsByIds([tweet_id])
}

In [ ]:
foo3 = await getTweetByUrl('https://twitter.com/phallusoraptors/status/1461964695556943879?s=20')

In [ ]:
last_word = foo3.data[0].text.trim().split(/\s+/).at(-1)
last_word.startsWith('https://t.co/')

In [ ]:
response = await axios.get(last_word, {maxRedirects: 0, validateStatus: (status) => status == 301})
url = response.headers.location
match = url.match(new RegExp('^https://twitter.com/(?<username>.+?)/status/(?<tweet_id>\\d+)$'))

In [ ]:
if (!match)

In [ ]:
[match.groups.username, match.groups.tweet_id]

In [ ]:
// https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
// since_id and start_time
// kinda useful, if wanna start processing from where left off last time?

In [ ]:
{ subDays, startOfToday, isBefore } = require('date-fns')

In [ ]:
start = subDays(startOfToday(), 2)

timeline = await client.v2.userTimeline(askyatharth, {'max_results': 100, 'tweet.fields': 'created_at', start_time: start.toISOString()})

In [ ]:
while (timeline.meta.next_token) {
    await timeline2.fetchNext()
}

In [ ]:
// TODO: handle rate limit error?

In [ ]:
// last_date = new Date(timeline.tweets.at(-1).created_at) 
// isBefore(last_date, startOfYesterday) || timeline3._realData.meta.next_token

## TODOs

In [3]:
// TODO: Protecter user? Or other error?
// TODO: How to only process until two days ago

In [ ]:
// TODO: Error handling! catch

// Uncaught ApiResponseError: Request failed with code 401
//     at RequestHandlerHelper.createResponseError (/Users/yatharth/Documents/misc/websites/quoterot/quoterot-cdk/node_modules/twitter-api-v2/dist/client-mixins/request-handler.helper.js:70:16)
//     at RequestHandlerHelper.onResponseEndHandler (/Users/yatharth/Documents/misc/websites/quoterot/quoterot-cdk/node_modules/twitter-api-v2/dist/client-mixins/request-handler.helper.js:110:25)
//     at IncomingMessage.emit (node:events:402:35)
//     at IncomingMessage.emit (node:domain:537:15)
//     at endReadableNT (node:internal/streams/readable:1340:12)
//     at processTicksAndRejections (node:internal/process/task_queues:83:21)

// {
//   error: true,
//   type: 'response',
//   code: 401,
//   headers: {
//     date: 'Sat, 20 Nov 2021 05:55:13 GMT',
//     pragma: 'no-cache',
//     server: 'tsa_a',
//     status: '401 Unauthorized',
//     expires: 'Tue, 31 Mar 1981 05:00:00 GMT',
//     'set-cookie': [
//       'guest_id_marketing=v1%3A163738771384305552; Max-Age=63072000; Expires=Mon, 20 Nov 2023 05:55:13 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None',
//       'guest_id_ads=v1%3A163738771384305552; Max-Age=63072000; Expires=Mon, 20 Nov 2023 05:55:13 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None',
//       'personalization_id="v1_C2cK9nZ2gLbLwYVNaSShcw=="; Max-Age=63072000; Expires=Mon, 20 Nov 2023 05:55:13 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None',
//       'guest_id=v1%3A163738771384305552; Max-Age=63072000; Expires=Mon, 20 Nov 2023 05:55:13 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None'
//     ],
//     'content-type': 'application/json;charset=utf-8',
//     'cache-control': 'no-cache, no-store, must-revalidate, pre-check=0, post-check=0',
//     'last-modified': 'Sat, 20 Nov 2021 05:55:13 GMT',
//     'x-transaction': '88fec3f5ad7947dc',
//     'content-length': '75',
//     'x-access-level': 'read',
//     'x-frame-options': 'SAMEORIGIN',
//     'www-authenticate': 'OAuth realm="https://api.twitter.com"',
//     'x-xss-protection': '0',
//     'x-rate-limit-limit': '1500',
//     'x-rate-limit-reset': '1637388613',
//     'content-disposition': 'attachment; filename=json.json',
//     'x-app-rate-limit-limit': '100000',
//     'x-app-rate-limit-reset': '1637458750',
//     'x-content-type-options': 'nosniff',
//     'x-rate-limit-remaining': '1499',
//     'x-twitter-response-tags': 'BouncerCompliant',
//     'strict-transport-security': 'max-age=631138519',
//     'x-app-rate-limit-remaining': '99991',
//     'x-response-time': '16',
//     'x-connection-hash': 'd3bd375e979a2b338b93ecec5a12e609d5374f49353d189f069c38fe6c96d538',
//     connection: 'close'
//   },
//   rateLimit: { limit: 1500, remaining: 1499, reset: 1637388613 },
//   data: {
//     request: '/1.1/statuses/user_timeline.json',
//     error: 'Not authorized.'
//   }
// }

In [ ]:
import { ApiResponseError } from 'twitter-api-v2';

try {
  // Get a single tweet
  await client.v1.tweet('20');
} catch (error) {
  if (error instanceof ApiResponseError && error.rateLimitError && error.rateLimit) {
    console.log(`You just hit the rate limit! Limit for this endpoint is ${error.rateLimit.limit} requests!`);
    console.log(`Request counter will reset at timestamp ${error.rateLimit.reset}.`);
  }
}